<a href="https://colab.research.google.com/github/HwangWooJin1028/YoloAIModel/blob/main/colab/potHole%ED%83%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 포트홀 탐지 모델 만들기
## 포트홀이란
  1. 포트홀은 도로에 만들어진 크랙의 종류로 깊게 패인 웅덩이를 말한다.
  2. 포트홀을 탐지하는 목적은
    - 자율 주행 자동차의 안전을 위한 회피운행
    - 도로 보수를 위한 위치 탐색

## 응용 SW를 개발하는 방법
  1. What ? : 위의 정의와 같이 무엇을 할 것인지를 정한다.
  2. DataSet을 준비한다. : 학습을 위한 addnotation 된 dataSet을 준비한다.
  3. 적합한 Pre-Trained된 모델을 구한다. :  이미지 관련서 우리는 YOLO에 집중한다.
  4. Fine-Tunning : 2에서 준비한 dataSet을 이용하여 custom Model을 만든다.
  5. 응용 SW를 제작한다.
    - webAPP : 실습한 TM을 이용한 JS를 기억한다.
    - nativeAPP : python과 pyQT5를 이용한 배포가능한 setup.exe 까지 완성한 것을 기억한다.
    

## YOLO git clone

In [ ]:
# 현재 폴더
%pwd
# 위치 이동을 강제로 해야한다면
%cd /content/
%pwd

/content


'/content'

In [ ]:
# github 에서 YOLO 가져오기
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16572, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 16572 (delta 26), reused 32 (delta 13), pack-reused 16522
Receiving objects: 100% (16572/16572), 15.10 MiB | 10.49 MiB/s, done.
Resolving deltas: 100% (11383/11383), done.


# 작업 폴더로 이동

In [ ]:
%cd yolov5/

/content/yolov5


In [ ]:
# dataSet을 가져올 폴더를 만든다
%mkdir /content/yolov5/pothole
%cd /content/yolov5/pothole

/content/yolov5/pothole


## dataSet 가져오기
  - data를 cul 명령어를 통하여 roboflow에서 가져온다.

In [ ]:
!curl -L "https://public.roboflow.com/ds/AZZeoZCGOT?key=XFPYdCUU9Z" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%pwd

'/content/yolov5/pothole'

## dataSet의 구성 내용 확인
*   각각의 데이터 셋이 얼마나 있는지 print한다.
    - images에 있는 pothole의 위치를 정규화한다.


In [ ]:
from glob import glob
train_img_list = glob('/content/yolov5/pothole/train/images/*.jpg')
test_img_list = glob('/content/yolov5/pothole/test/images/*.jpg')
valid_img_list = glob('/content/yolov5/pothole/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

465 67 133


*   glob
    - 파일 경로를 매칭하여 파일들의 리스트를 반환하는데 사용
    - 디렉토리 내의 파일을 다룰 때 사용

# 모델 FineTunning 시작

## 1. dataSet의 위치등을 알려주기 위한 yaml 파일 준비

### 위에서 만든 list를 txt 파일로 저장한다.


In [ ]:
import yaml
with open('/content/yolov5/pothole/train.txt','w') as f:
    f.write('\n'.join(train_img_list) + '\n')
with open('/content/yolov5/pothole/test.txt','w') as f:
    f.write('\n'.join(test_img_list) + '\n')
with open('/content/yolov5/pothole/val.txt','w') as f:
    f.write('\n'.join(valid_img_list) + '\n')

*   yaml
    - YAML 형식의 데이터를 파이썬 객체로 읽거나 파이썬 객체를 yaml 형식으로 쓰는 데 사용
    - yaml: 사람이 쓰기 쉬운 데이터 직렬화 방식

### fineTunning 중에서 자주 사용하는 writemplate 함수를 정의

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line,cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

### FineTunning 중에 모델에게 dataSet의 위치를 알려주기 위한 data.yaml을 만든다.

In [ ]:
%%writetemplate /content/yolov5/pothole/data.yaml

train: ./pothole/train/images
test: ./pothole/test/images
val: ./pothole/valid/images

nc: 1
names: ['pothole']

In [ ]:
%cat /content/yolov5/pothole/data.yaml


train: ./pothole/train/images
test: ./pothole/test/images
val: ./pothole/valid/images

nc: 1
names: ['pothole']


### 만들어진 data.yaml에서 필요한 변수(nc)를 가져오기

In [ ]:
with open('/content/yolov5/pothole/data.yaml','r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])
  print(num_classes)


1


### custom.yolov5s.yaml 만들기
    1. 원래 pre-trained된 yolov5s.yaml이 갖고 있는 내용을 수정해서 만들어야 함
      - 원본 yolov5s.yaml을 cat으로 프린트
      - 복사하여 사용

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 

### custom_yolov5s.yaml 파일을 만든다

    * nc : 80 또는 nc : 다른 숫자
    부분을 앞서 가져온 변수 num_classes로 바꾸어 준다. {num_classes}

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

### 최종 만들어진 custom_yolov5s.yaml의 내용 중 nc가 수정되었는지 확인한다
    nc = numclasses = 1

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml


# Parameters
nc: 1  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, Fa

## 2. 모델 학습 시작

### 학습에 필요한 parameter 지정하기
    - 우리가 사용하는 tarin.py는 yoloV5 모델을 발표한 사람이 직접 작성한 코드이다.
    - 우리는 1줄의 수정없이 코드를 그대로 사용하여 fine Tunning 진행
    - 다만 parameter를 이용하여 작업 방법을 알려준다

#### parameter 종류
    - `img`: 입력 이미지 크기 정의
    - `batch`: 배치 크기 결정 (한번 읽어올때마다 이 크기만큼 읽어옴)
    - `epochs`: 학습 기간 개수 정의
    - `data`: yaml 파일 경로
    - `cfg`: 모델 구성 지정
    - `weights`: 가중치에 대한 경로 지정
    - `name`: 결과 이름
    - `nosave`: 최종 체크포인트만 저장
    - `cache`: 빠른 학습을 위한 이미지 캐시

### 학습을 위한 시간 기록 및 폴더 이동

In [ ]:
%%time
%cd /content/yolov5/

/content/yolov5
CPU times: user 1.65 ms, sys: 0 ns, total: 1.65 ms
Wall time: 1.43 ms


### train.py에 parameter 전달하고 실행

#### 아래 코드를 실행 중 error가 발생 할 수 있어요
    1. 원인은 ultralytics의 gitpython 모듈이 설치가 안되어
    2. 자동 설치 하고 다시 restart 되면서
    3. /content/yolov5/pothole/train/labels.cache 가 만들어졌기 때문
    4. 이 파일을 제거하고 코드 다시 재실행

In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data ./pothole/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name pothole_result --cache

성적이 매우 안좋은 것을 확인할 수 있다.

#### 2. tensorboard를 이용한 확인

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

    3. 학습 과정 중에 validation data를 이용하여 스스로 검증한 결과를 이미지로 출력한 내용을 확인할 수 있다.

In [ ]:
!ls /content/yolov5/runs/train/pothole_result2

confusion_matrix.png				    PR_curve.png	   val_batch0_pred.jpg
events.out.tfevents.1714360216.fee50e28cbff.2004.0  R_curve.png		   val_batch1_labels.jpg
F1_curve.png					    results.csv		   val_batch1_pred.jpg
hyp.yaml					    results.png		   val_batch2_labels.jpg
labels_correlogram.jpg				    train_batch0.jpg	   val_batch2_pred.jpg
labels.jpg					    train_batch1.jpg	   weights
opt.yaml					    train_batch2.jpg
P_curve.png					    val_batch0_labels.jpg


## 모델 검증


### 실제로 검증 내용은 앞서 확인한 텐서보드의 내용과 동일

    1. 가중치 파일은 우리가 파인튜닝한 것을 사용한다.
    2. /content/yolov5/runs/train/pothole_result2/weights/best.pt는 학습 과정 중 가장 성적이 좋은 것이다

In [ ]:
!python val.py --weights /content/yolov5/runs/train/pothole_result2/weights/best.pt --data /content/yolov5/pothole/data.yaml --img 640 --iou 0.65 --half

val: data=/content/yolov5/pothole/data.yaml, weights=['/content/yolov5/runs/train/pothole_result2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-306-gb599ae42 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/yolov5/pothole/valid/labels.cache... 133 images, 0 backgrounds, 0 corrupt: 100% 133/133 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances     

## 추론
    1. 이제는 실제 사용하기 위하여 시뮬레이션을 시작
    2. test 데이터를 이용한다.
    3. 앞서 yolov5를 사용해서 Object detection을 했던 것 처럼 이번에는 우리가 만든 모델을 이용해서 Object detection을

파인튜닝한 결과물이 저장된 (best.py) 위치를 확인해서 폴더 위치를 지정해야 한다

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/pothole_result2/weights/best.pt --img 640 --conf 0.4 --source ./pothole/test/images

### 결과물 확인

In [ ]:
import glob
import random
from IPython.display import Image, display

image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp/img-105_jpg.rf.3fe9dff3d1631e79ecb480ff403bcb86.jpg'))

display(Image(filename=image_name))

## 구글 드라이브 혹은 내 컴퓨터에 다운로드

  1. 구글드라이브에 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%mkdir /content/drive/MyDrive/pothole
%cp /content/yolov5/runs/train/pothole_result2/weights/best.pt /content/drive/MyDrive/pothole

  2. 바로 다운로드
    - yolov5 > runs > detect > train > pothole_result2 > weights > best.pt 다운로드